#### Collection of code blocks

* Get Azure Oauth to use desired Azure Subscription
* Create Azure ML Service Workspace instance
* Register available model files into Azure ML Service Workspace as Models which will be used during Prediction Serving
* Scoring code to be emitted into 'score.py'
* Create Docker Image, deployment of container instances, deployment of Azure ML Web Service for scoring/prediction

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
import pickle, json

In [3]:
# Ensure to use the correct Azure Subscription
# As this work is being initiated from Jupyter Notebook from Azure portal, it needs to be Authorized through Device Assisted Azure Login

!az account set -s <subscription-id>
!az login

# You can expect to see this below instruction for completing Azure login & oauth
#
# To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F2K6SGE5K to authenticate.
#

In [2]:
# Workspace instance is being created from config.json (which is expected to have subscription-id, resource-group-name & azure-ml-service-workspace-name)

ws = Workspace.from_config()

NameError: name 'Workspace' is not defined

In [8]:
# Below 2 models are from offline ML Model Training in .pkl format
# Here these model files are being registered as Model in Azure ML Service Workspace

model = Model.register(model_path = "classifier_model.pkl",
                       model_name = "classifier_model",
                       tags = {"key": "0.1"},
                       description = "classifier_model",
                       workspace = ws)

vocab = Model.register(model_path = "vocab.pkl",
                       model_name = "vocab",
                       tags = {"key": "0.1"},
                       description = "vocab",
                       workspace = ws)

Registering model classifier_model
Registering model vocab


In [16]:
# Scoring code to be deployed in Azure ML Service and it will be used to serve Prediction through REST Web Service

%%writefile score.py
import pickle, json
import numpy as np
import pandas as pd
import re

from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short

from sklearn.feature_extraction.text import CountVectorizer

from azureml.core.model import Model


def init():
    global model, vocab, txt_filters
    
    txt_filters = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short]
    
    model_path = Model.get_model_path(model_name = "classifier_model")
    with open(model_path, "rb") as f_model:
        model = pickle.load(f_model)
        
    vocab_path = Model.get_model_path(model_name = "vocab")
    with open(vocab_path, "rb") as f_vocab:
        vocab = pickle.load(f_vocab)

        
def process_input(row):
    input_merged = row['Assignment Name'] + ' ' + row['School Category']
    
    # gensim's preprocess_string through series of txt_filters which generates tokens array
    input_processed_tokens = " ".join(preprocess_string(input_merged, txt_filters))
    
    return input_processed_tokens


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

    
def run(raw_data):
    try:
        input_json = json.loads(raw_data)
        
        df = pd.DataFrame.from_dict(input_json, orient='columns')
        df['processed_input'] = df.apply(lambda row: process_input(row), axis=1)
        
        count_vect = CountVectorizer(vocabulary=vocab)
        count_vect._validate_vocabulary()
        
        prediction = model.predict(count_vect.transform(df['processed_input']))
        
        # Overriding ML Model predicted label with rule-based decision
        labels_dict = {}
        labels_dict['assignment'] = 0
        labels_dict['quiz'] = 1
        labels_dict['homework'] = 2
        labels_dict['test'] = 3
        labels_dict['extra credit'] = 4

        arr_labels = ['assignment', 'quiz', 'homework', 'test', 'extra credit']

        for index, row in df.iterrows():
            label_match_school_category = re.search('assignment|quiz|homework|test|extra credit', row['School Category'].lower())
            label_match_assignment_name = re.search('assignment|quiz|homework|test|extra credit', row['Assignment Name'].lower()) 
            predicted_match_school_category = re.search(arr_labels[prediction[index]], row['School Category'].lower())
            predicted_match_assignment_name = re.search(arr_labels[prediction[index]], row['Assignment Name'].lower())     
            if label_match_school_category and (label_match_assignment_name is None) and (predicted_match_school_category is None):
                prediction[index] = labels_dict[label_match_school_category.group()]
            elif label_match_assignment_name and (label_match_school_category is None) and (predicted_match_assignment_name is None):
                prediction[index] = labels_dict[label_match_assignment_name.group()]
        
        out_json = json.dumps(prediction, cls=NumpyEncoder)
        return out_json
    
    except Exception as e:
        msg_exception = str(e)
        return json.dumps({"error": msg_exception})

Overwriting score.py


#### Below code blocks actually perform the following steps
* Create Docker Container Images with required Conda, Python & Dependencies
* Creates 'text-classifier' Deployment instance - this actually points to ACI (Azure Container Instances) managed multiple Container Instances
* Deploys 'text-classifier' Azure ML Web Service

In [10]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("gensim")
myenv.save_to_file(".", "myenv.yml")

'myenv.yml'

In [17]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

image_config = ContainerImage.image_configuration(execution_script = "score.py", 
                                    runtime = "python", 
                                    conda_file = "myenv.yml")

service = Webservice.deploy_from_model(name = "text-classifier",
                                       deployment_config = aci_config,
                                       models = [model, vocab],
                                       image_config = image_config,
                                       workspace = ws)

service.wait_for_deployment(show_output = True)

Creating image
Image creation operation finished for image text-classifier:5, operation "Succeeded"
Creating service
Running......................
SucceededACI service creation operation finished, operation "Succeeded"


In [1]:
# Fetching Scoring URL of the deployed Azure ML Web Service

from azureml.core.webservice import Webservice

services = Webservice.list(ws)
print(services[0].scoring_uri)

ModuleNotFoundError: No module named 'azureml'

In [ ]:
# Alternate for fetching Scoring/Prediction URL of the deployed Azure ML Web Service

service = Webservice(workspace=ws, name='text-classifier')
print(service.scoring_uri)